# Importing libraries

In [2]:
import pandas as pd     # for calculation
import numpy as np

from sklearn.experimental import enable_iterative_imputer   # for mssing values calculation
from sklearn.impute import IterativeImputer
from sklearn.ensemble import IsolationForest

import matplotlib.pyplot as plt     # for graph

from sklearn.cluster import DBSCAN  # for outliers

# User-Defined Function (UDF)

In [3]:
# creating a function for monthly calculation from yearly and weakly 
def monthly(x, value):
    if x in [409,419, 619, 649]:
        return (value * 30)/ 365
    elif x in [169, 219, 239, 249, 189, 269, 279, 199, 289, 299, 309, 319, 329]:
        return (value * 30)/ 7
    else:
        return value

In [4]:
# creating a function for sector and state spliting from Common-id
def split_string(s):
    part0 = s[:13]   # first 13 bytes
    part1 = s[13:14]  # next 2 bytes
    part2 = s[14:16]  # next 2 bytes
    part3 = s[16:]   # remaining bytes
    return pd.Series([part0,part1, part2, part3]) 

# Importing data

In [5]:
# Importing level 2 data from divice
col_specification=[(0,38),(38,39),(39,41),(41,43),(43,44),(44,45),(45,48),(48,49),(49,51),(51,53),(53,54),(54,56),(56,57),(57,59),(59,61),(61,63),(63,65),(65,67),(67,68),(68,69),(69,84)]
col_names=['Common-ID','Questionnaire No.','Level','Person Srl No.','Relation to head (code)','Gender','Age(in years)','Marital Status (code)','Highest educational level attained (code)','Total year of education completed','Whether used internet from any location during last 30 days','No. of days stayed away from home during last 30 days','No. of meals usually taken in a day','No. of meals taken during last 30 days from school, balwadi etc.','No. of meals taken during last 30 days  others','No. of meals taken during last 30 days from employer as perquisites or part of wage','No. of meals taken during last 30 days at home','No. of meals taken during last 30 days on payment','FDQ original member(generated field)','Status of Member as on revisit','Multiplier']
level_2 = pd.read_fwf(r'F:\DATA\D_work\Interpreation\hces22_lvl_02.txt', header=None, colspecs=col_specification, names = col_names)

# As provided in "Read me" text dividing multiplier by 100
level_2['Multiplier'] = level_2['Multiplier']/100
level_2.head(5)

,Common-ID,Questionnaire No.,Level,Person Srl No.,Relation to head (code),Gender,Age(in years),Marital Status (code),Highest educational level attained (code),Total year of education completed,...,No. of days stayed away from home during last 30 days,No. of meals usually taken in a day,"No. of meals taken during last 30 days from school, balwadi etc.",No. of meals taken during last 30 days others,No. of meals taken during last 30 days from employer as perquisites or part of wage,No. of meals taken during last 30 days at home,No. of meals taken during last 30 days on payment,FDQ original member(generated field),Status of Member as on revisit,Multiplier
0,HCES2022655561010131113011 101202 201,H,2,1,1,1,48,2,6,12.0,...,0.0,2.0,0.0,0.0,0.0,0.0,58.0,1,1.0,355.6
1,HCES2022655561010131113011 101202 201,H,2,2,2,2,46,2,1,NaN,...,0.0,2.0,NaN,NaN,NaN,NaN,60.0,1,1.0,355.6
2,HCES2022655561010131113011 101202 201,H,2,3,5,1,24,1,13,18.0,...,0.0,2.0,NaN,NaN,NaN,NaN,58.0,1,1.0,355.6
3,HCES2022655561010131113011 101202 201,H,2,4,5,1,18,1,7,13.0,...,0.0,2.0,NaN,NaN,NaN,NaN,56.0,1,1.0,355.6
4,HCES2022655561010131113011 101202 201,H,2,5,5,2,21,1,12,17.0,...,0.0,2.0,NaN,NaN,NaN,NaN,54.0,1,1.0,355.6


In [6]:
z =level_2.groupby('Common-ID')['Multiplier'].unique().reset_index()

In [7]:
z.head()

,Common-ID,Multiplier
0,HCES2022310002282831212025 228111 301,[964.98]
1,HCES2022310002282831212025 228111 302,[964.98]
2,HCES2022310002282831212025 228111 303,[964.98]
3,HCES2022310002282831212025 228111 304,[964.98]
4,HCES2022310002282831212025 228111 305,[964.98]


In [8]:
z[['part2','Sector', 'State', 'part3']] = z['Common-ID'].apply(split_string)

In [9]:
z.head()

,Common-ID,Multiplier,part2,Sector,State,part3
0,HCES2022310002282831212025 228111 301,[964.98],HCES202231000,2,28,2831212025 228111 301
1,HCES2022310002282831212025 228111 302,[964.98],HCES202231000,2,28,2831212025 228111 302
2,HCES2022310002282831212025 228111 303,[964.98],HCES202231000,2,28,2831212025 228111 303
3,HCES2022310002282831212025 228111 304,[964.98],HCES202231000,2,28,2831212025 228111 304
4,HCES2022310002282831212025 228111 305,[964.98],HCES202231000,2,28,2831212025 228111 305


In [10]:
# Importing level-14 data from device
col_specification=[(0,38),(38,39),(39,41),(41,46),(46,49),(49,59),(59,74)]
col_names=['Common-ID','Questionnaire No.','Level','Section','Item Code','Value (in Rs)','Multiplier']
lvl_14= pd.read_fwf(r'F:\DATA\D_work\Interpreation\hces22_lvl_14.txt', header=None, colspecs=col_specification, names = col_names)

# As provided in "Read me" text dividing multiplier by 100
lvl_14['Multiplier'] = lvl_14['Multiplier']/100
lvl_14.head(5)

,Common-ID,Questionnaire No.,Level,Section,Item Code,Value (in Rs),Multiplier
0,HCES2022655621010121713017 101111 301,F,14,5.1,139,90,88.9
1,HCES2022655621010121713017 101111 310,F,14,5.1,139,90,88.9
2,HCES2022655621010121713017 101111 311,F,14,5.1,139,25,88.9
3,HCES2022655621010121713017 101111 313,F,14,5.1,139,45,88.9
4,HCES2022655371010122023014 201111 202,F,14,5.1,139,75,1405.0


### Primary cleanning
Splitting the non-usable column and multiplier divided by 100 as defined in the "read me" word file

In [11]:
# for counting total number of members of a household
lvl_2 = level_2.value_counts('Common-ID').to_frame().reset_index()
lvl_2.head(5)

,Common-ID,count
0,HCES2022666371080811523175 108112 107,37
1,HCES2022341462363612342036 236101 1312,31
2,HCES2022636211090934223383 109205 303,30
3,HCES2022670621353510123017 235101 203,29
4,HCES2022636811090920723403 109125 308,28


In [12]:
mu = pd.merge(lvl_2,z, on='Common-ID',how='left')
mu.head()

,Common-ID,count,Multiplier,part2,Sector,State,part3
0,HCES2022666371080811523175 108112 107,37,[1852.97],HCES202266637,1,08,0811523175 108112 107
1,HCES2022341462363612342036 236101 1312,31,[1859.42],HCES202234146,2,36,3612342036 236101 1312
2,HCES2022636211090934223383 109205 303,30,[2087.25],HCES202263621,1,09,0934223383 109205 303
3,HCES2022670621353510123017 235101 203,29,[88.19],HCES202267062,1,35,3510123017 235101 203
4,HCES2022636811090920723403 109125 308,28,[1394.85],HCES202263681,1,09,0920723403 109125 308


In [31]:
len(mu)

261746

In [13]:
mu['total_mambers'] = mu['count']*mu['Multiplier']
mu.head(5)

,Common-ID,count,Multiplier,part2,Sector,State,part3,total_mambers
0,HCES2022666371080811523175 108112 107,37,[1852.97],HCES202266637,1,08,0811523175 108112 107,[68559.89]
1,HCES2022341462363612342036 236101 1312,31,[1859.42],HCES202234146,2,36,3612342036 236101 1312,[57642.020000000004]
2,HCES2022636211090934223383 109205 303,30,[2087.25],HCES202263621,1,09,0934223383 109205 303,[62617.5]
3,HCES2022670621353510123017 235101 203,29,[88.19],HCES202267062,1,35,3510123017 235101 203,[2557.5099999999998]
4,HCES2022636811090920723403 109125 308,28,[1394.85],HCES202263681,1,09,0920723403 109125 308,[39055.799999999996]


In [14]:
# Dropping level and Section from data frame (lvl_14)
level_14 = lvl_14.drop(['Level','Section'],axis=1)
level_14.head(5)

,Common-ID,Questionnaire No.,Item Code,Value (in Rs),Multiplier
0,HCES2022655621010121713017 101111 301,F,139,90,88.9
1,HCES2022655621010121713017 101111 310,F,139,90,88.9
2,HCES2022655621010121713017 101111 311,F,139,25,88.9
3,HCES2022655621010121713017 101111 313,F,139,45,88.9
4,HCES2022655371010122023014 201111 202,F,139,75,1405.0


In [15]:
main_data = pd.merge(level_14,mu, on='Common-ID', how='left')
main_data.head(5)

,Common-ID,Questionnaire No.,Item Code,Value (in Rs),Multiplier_x,count,Multiplier_y,part2,Sector,State,part3,total_mambers
0,HCES2022655621010121713017 101111 301,F,139,90,88.9,5,[88.9],HCES202265562,1,01,0121713017 101111 301,[444.5]
1,HCES2022655621010121713017 101111 310,F,139,90,88.9,5,[88.9],HCES202265562,1,01,0121713017 101111 310,[444.5]
2,HCES2022655621010121713017 101111 311,F,139,25,88.9,6,[88.9],HCES202265562,1,01,0121713017 101111 311,[533.4000000000001]
3,HCES2022655621010121713017 101111 313,F,139,45,88.9,5,[88.9],HCES202265562,1,01,0121713017 101111 313,[444.5]
4,HCES2022655371010122023014 201111 202,F,139,75,1405.0,4,[1405.0],HCES202265537,1,01,0122023014 201111 202,[5620.0]


In [32]:
len(main_data)

8075528

In [16]:
main_data['total_mambers'] = main_data['total_mambers'].astype(int)
main_data.head(5)

,Common-ID,Questionnaire No.,Item Code,Value (in Rs),Multiplier_x,count,Multiplier_y,part2,Sector,State,part3,total_mambers
0,HCES2022655621010121713017 101111 301,F,139,90,88.9,5,[88.9],HCES202265562,1,01,0121713017 101111 301,444
1,HCES2022655621010121713017 101111 310,F,139,90,88.9,5,[88.9],HCES202265562,1,01,0121713017 101111 310,444
2,HCES2022655621010121713017 101111 311,F,139,25,88.9,6,[88.9],HCES202265562,1,01,0121713017 101111 311,533
3,HCES2022655621010121713017 101111 313,F,139,45,88.9,5,[88.9],HCES202265562,1,01,0121713017 101111 313,444
4,HCES2022655371010122023014 201111 202,F,139,75,1405.0,4,[1405.0],HCES202265537,1,01,0122023014 201111 202,5620


In [17]:
level_14.count()

Common-ID            8075528
Questionnaire No.    8075528
Item Code            8075528
Value (in Rs)        8075528
Multiplier           8075528
dtype: int64

## Appling monthly function
so we can read monthly expendeture

In [18]:
# Creating a new column for monthly calculated values using monthly function
main_data['Monthly Value'] = main_data.apply(lambda row: monthly(row['Item Code'], row['Value (in Rs)']), axis=1)
main_data.head(5)

,Common-ID,Questionnaire No.,Item Code,Value (in Rs),Multiplier_x,count,Multiplier_y,part2,Sector,State,part3,total_mambers,Monthly Value
0,HCES2022655621010121713017 101111 301,F,139,90,88.9,5,[88.9],HCES202265562,1,01,0121713017 101111 301,444,90.0
1,HCES2022655621010121713017 101111 310,F,139,90,88.9,5,[88.9],HCES202265562,1,01,0121713017 101111 310,444,90.0
2,HCES2022655621010121713017 101111 311,F,139,25,88.9,6,[88.9],HCES202265562,1,01,0121713017 101111 311,533,25.0
3,HCES2022655621010121713017 101111 313,F,139,45,88.9,5,[88.9],HCES202265562,1,01,0121713017 101111 313,444,45.0
4,HCES2022655371010122023014 201111 202,F,139,75,1405.0,4,[1405.0],HCES202265537,1,01,0122023014 201111 202,5620,75.0


# One item study
For checking data is representing correctly or not we only study 'Item code' 169 for milk and milk products 

In [19]:
# Filtering the data-frame for checking milk & milk prodect consumption in a single housee hold
##checking if monthly funtion is functioning properly
todoroki = main_data[main_data['Item Code'].isin([409])]
temp1 = todoroki.copy()
temp1.head(5)

,Common-ID,Questionnaire No.,Item Code,Value (in Rs),Multiplier_x,count,Multiplier_y,part2,Sector,State,part3,total_mambers,Monthly Value
3649908,HCES2022655561010131113011 101202 201,C,409,24050,355.60,5,[355.6],HCES202265556,1,01,0131113011 101202 201,1778,1976.712329
3649915,HCES2022655561010131113011 101202 301,C,409,30500,303.31,6,[303.31],HCES202265556,1,01,0131113011 101202 301,1819,2506.849315
3649922,HCES2022655561010131113011 101202 302,C,409,27100,303.31,8,[303.31],HCES202265556,1,01,0131113011 101202 302,2426,2227.397260
3649929,HCES2022655561010131113011 101202 303,C,409,16250,303.31,4,[303.31],HCES202265556,1,01,0131113011 101202 303,1213,1335.616438
3649954,HCES2022655561010131113011 101202 307,C,409,22800,303.31,5,[303.31],HCES202265556,1,01,0131113011 101202 307,1516,1873.972603


In [20]:
# description of monthly value only for milk & milk prodects ('4099') 
temp1['total_Value'] = temp1['Monthly Value']*temp1['Multiplier_x']
temp1.head(5)

,Common-ID,Questionnaire No.,Item Code,Value (in Rs),Multiplier_x,count,Multiplier_y,part2,Sector,State,part3,total_mambers,Monthly Value,total_Value
3649908,HCES2022655561010131113011 101202 201,C,409,24050,355.60,5,[355.6],HCES202265556,1,01,0131113011 101202 201,1778,1976.712329,702918.904110
3649915,HCES2022655561010131113011 101202 301,C,409,30500,303.31,6,[303.31],HCES202265556,1,01,0131113011 101202 301,1819,2506.849315,760352.465753
3649922,HCES2022655561010131113011 101202 302,C,409,27100,303.31,8,[303.31],HCES202265556,1,01,0131113011 101202 302,2426,2227.397260,675591.863014
3649929,HCES2022655561010131113011 101202 303,C,409,16250,303.31,4,[303.31],HCES202265556,1,01,0131113011 101202 303,1213,1335.616438,405105.821918
3649954,HCES2022655561010131113011 101202 307,C,409,22800,303.31,5,[303.31],HCES202265556,1,01,0131113011 101202 307,1516,1873.972603,568394.630137


In [21]:
temp_dropped = temp1.drop(['Questionnaire No.','Item Code','Multiplier_x','count','Multiplier_y','part2','part3'],axis=1)
temp_dropped.head(5)

,Common-ID,Value (in Rs),Sector,State,total_mambers,Monthly Value,total_Value
3649908,HCES2022655561010131113011 101202 201,24050,1,01,1778,1976.712329,702918.904110
3649915,HCES2022655561010131113011 101202 301,30500,1,01,1819,2506.849315,760352.465753
3649922,HCES2022655561010131113011 101202 302,27100,1,01,2426,2227.397260,675591.863014
3649929,HCES2022655561010131113011 101202 303,16250,1,01,1213,1335.616438,405105.821918
3649954,HCES2022655561010131113011 101202 307,22800,1,01,1516,1873.972603,568394.630137


In [23]:
temp_dropped['per_person_per_month'] = temp_dropped['total_Value']/temp_dropped['total_mambers']
temp_dropped.head(5)

,Common-ID,Value (in Rs),Sector,State,total_mambers,Monthly Value,total_Value,per_person_per_month
3649908,HCES2022655561010131113011 101202 201,24050,1,01,1778,1976.712329,702918.904110,395.342466
3649915,HCES2022655561010131113011 101202 301,30500,1,01,1819,2506.849315,760352.465753,418.005754
3649922,HCES2022655561010131113011 101202 302,27100,1,01,2426,2227.397260,675591.863014,278.479746
3649929,HCES2022655561010131113011 101202 303,16250,1,01,1213,1335.616438,405105.821918,333.970175
3649954,HCES2022655561010131113011 101202 307,22800,1,01,1516,1873.972603,568394.630137,374.930495


In [30]:
temp_dropped.describe()

,Value (in Rs),total_mambers,Monthly Value,total_Value,per_person_per_month
count,1.560890e+05,156089.000000,156089.000000,1.560890e+05,156089.000000
mean,1.796430e+04,5347.393263,1476.518196,1.519066e+06,377.565023
std,3.322612e+04,7259.551899,2730.914086,4.516711e+06,1103.431204
min,1.000000e+00,11.000000,0.082192,3.540822e+01,0.082205
25%,2.250000e+03,2136.000000,184.931507,1.199905e+05,38.001876
50%,8.000000e+03,4791.000000,657.534247,4.648987e+05,136.445111
75%,2.119000e+04,7284.000000,1741.643836,1.465928e+06,382.233923
max,3.078000e+06,966006.000000,252986.301370,3.627697e+08,91065.610932


In [29]:
temp_dropped['Sector'].unique()

array(['1', '2'], dtype=object)

In [25]:
Urban_edu = temp_dropped[temp_dropped['Sector'].isin(['1'])]

In [27]:
Urban_edu.describe()

,Value (in Rs),total_mambers,Monthly Value,total_Value,per_person_per_month
count,9.312400e+04,93124.000000,93124.000000,9.312400e+04,93124.000000
mean,1.116863e+04,6473.405814,917.969753,1.165901e+06,197.929798
std,2.130931e+04,8646.483857,1751.449804,3.742422e+06,514.446250
min,4.000000e+00,11.000000,0.328767,4.075890e+01,0.091329
25%,1.550000e+03,3899.000000,127.397260,1.089321e+05,25.481591
50%,5.080000e+03,5998.000000,417.534247,3.891979e+05,83.671919
75%,1.315000e+04,8265.000000,1080.821918,1.190900e+06,216.618370
max,1.400000e+06,966006.000000,115068.493151,2.202612e+08,57538.943249


In [28]:
Rural_edu = temp_dropped[temp_dropped['Sector'].isin(['2'])]
Rural_edu.describe()

,Value (in Rs),total_mambers,Monthly Value,total_Value,per_person_per_month
count,6.296500e+04,62965.000000,62965.000000,6.296500e+04,62965.000000
mean,2.801497e+04,3682.042786,2302.600387,2.041392e+06,643.241998
std,4.354137e+04,3927.583541,3578.742372,5.422339e+06,1583.857457
min,1.000000e+00,12.000000,0.082192,3.540822e+01,0.082205
25%,5.350000e+03,1074.000000,439.726027,1.436888e+05,97.428134
50%,1.600000e+04,2921.000000,1315.068493,6.265393e+05,298.636977
75%,3.500000e+04,5012.000000,2876.712329,1.992616e+06,682.372054
max,3.078000e+06,137426.000000,252986.301370,3.627697e+08,91065.610932
